In [1]:
import gym
import pybullet_envs
import numpy as np
import neat
import pickle
import random
import signal
import matplotlib.pyplot as plt

num_cores = 8

random.seed(0)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'data/config-humanoid')
p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True)) 
p.add_reporter(neat.StatisticsReporter())
p.add_reporter(neat.Checkpointer(1, None, "saves/robotic_checkpoint_"))

In [2]:
# for parallel evaluating:
class Worker(object):
    def __init__(self, genome, config):
        self.genome = genome
        self.config = config
        
    def work(self):

        self.env = gym.make('AntBulletEnv-v0')
        self.env.seed(0)
        ob = self.env.reset()

        done = False

        net = neat.nn.recurrent.RecurrentNetwork.create(self.genome, self.config)
        
        current_max_fitness = 0
        fitness_current = 0
        frame = 0
        counter = 0

        finallyCounter = 1000
        
        # SIGINT is sent by the stop-button in jupyter
        signal.signal(signal.SIGINT, lambda signal, frame: globals().update(done=True))

        while not done:
            
            frame += 1

            nnOutput = net.activate(ob)

            ob, rew, done, info = self.env.step(nnOutput)
            
            fitness_current += rew
            
            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
                
            finalReached = (counter>=finallyCounter)
            
                
            if done or finalReached:
                done = True
                print('fitness: '+str(fitness_current)+' counter: '+str(counter))
                
        return fitness_current

In [3]:
def eval_genomes(genome, config):
    worky = Worker(genome, config)
    return worky.work()

In [4]:
def plot_stats(statistics, ylog=False, view=False, filename='avg_fitness.svg'):
    """ Plots the population's average and best fitness. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    generation = range(len(statistics.most_fit_genomes))
    best_fitness = [c.fitness for c in statistics.most_fit_genomes]
    avg_fitness = np.array(statistics.get_fitness_mean())
    stdev_fitness = np.array(statistics.get_fitness_stdev())

    plt.plot(generation, avg_fitness, 'b-', label="average")
    plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, avg_fitness + stdev_fitness, 'g-.', label="+1 sd")
    plt.plot(generation, best_fitness, 'r-', label="best")

    plt.title("Population's average and best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid()
    plt.legend(loc="best")
    if ylog:
        plt.gca().set_yscale('symlog')

    plt.savefig(filename)
    if view:
        plt.show()

    plt.close()

In [5]:
pe = neat.ParallelEvaluator(num_cores, eval_genomes)

winner = p.run(pe.evaluate)

with open('winnerPara.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)
    
#with open('winnerPara.pkl','rb') as readWinner:
#    winner = pickle.load(readWinner)
#    print('\nBest genome:\n{!s}'.format(winner))
#    plot_stats(stats, ylog=False, view=True)


 ****** Running generation 0 ****** 

fitness: 2.978677166574927 counter: 1
fitness: -1.0131154060758913 counter: 1
fitness: 68.52742562577777 counter: 1
fitness: 431.1938240062727 counter: 0
fitness: 308.97476942295606 counter: 0
fitness: 209.7999475541842 counter: 0
fitness: 260.152549832649 counter: 0
fitness: 289.5304382573998 counter: 0
fitness: 291.94699164632755 counter: 0
fitness: 454.92986338692253 counter: 0
fitness: 359.440236587919 counter: 0
fitness: 432.8064247028425 counter: 0
fitness: 4.796499629317421 counter: 1
fitness: 138.93826510049507 counter: 0
fitness: 309.32480528246117 counter: 0
fitness: 322.41348354798276 counter: 0
fitness: 274.6438120477439 counter: 0
fitness: 600.3752671260219 counter: 0
fitness: 171.89882254795083 counter: 0
fitness: 229.1348467125579 counter: 0
Population's average fitness: 258.03969 stdev: 157.40660
Best fitness: 600.37527 - size: (8, 116) - species 3 - id 15
Average adjusted fitness: 0.389
Mean genetic distance 2.305, standard deviat

KeyboardInterrupt: 

fitness: 697.631047588092 counter: 0
fitness: 567.1997629912737 counter: 0
fitness: 374.03924182927125 counter: 0
fitness: 615.0879747399889 counter: 0
